In [1]:
from pathlib import Path
import pandas as pd

import emoji
import preprocessor as p
import truecase

p.set_options(p.OPT.URL)

# Preprocessing Function

In [2]:
def lowercasing(sentence):
    return str(sentence).lower()

def truecasing(sentence):
    """ Alternative function to 'lowercasing().
        Instead of turning everything into lower case, the grammatically true case
        is used."""
    return truecase.get_true_case(str(sentence))

def rm_whitespace(sentence):
    return str(sentence).strip()

def replace_emoji(sentence):
    return emoji.demojize(str(sentence))

def replace_url(sentence):
    return p.tokenize(str(sentence))

In [11]:
def preprocessing(s):
    #s = lowercasing(s)
    s = truecasing(s)
    s = rm_whitespace(s)
    s = replace_emoji(s)    
    s = replace_url(s)
    return s


#e.g.
s = ' Preprocessor is #awesome 👍 https://github.com/s/preprocessor '
preprocessing(s)

'Feliz Año Nuevo, Feliz Esperanza :two_hearts: . Querido 2023, ¡Allá Vamos!.'

# Files cleaning

In [6]:
datafolder = Path("./data/full_data/")
print(list(datafolder.rglob('*.csv')))

train_f= Path('data/full_data/full_data.csv')
# test_f= Path('data/full_data/development_test.csv')

[PosixPath('data/full_data/full_data.csv')]


In [15]:
def clean_df(df, f_path):
    df["cleaned_tweet"] = df["tweet"].apply(lambda s: preprocessing(s))

    df_path = f_path.parent/"cleaned"/f"cleaned_{f_path.name}"
    df_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(df_path)
    return train_df



In [16]:
train_df = pd.read_csv(train_f)
# test_df = pd.read_csv(test_f)

In [17]:
clean_df(train_df,train_f)
# clean_df(test_df,test_f)

,label,gender,profession,ideology_binary,ideology_multiclass,tweet,cleaned_tweet
0,0008c4fab9e97623a60380ee9c88cb20,female,politician,left,left,¡Feliz 28 de febrero a todas las andaluzas y a...,¡Feliz 28 de Febrero a Todas Las Andaluzas Y A...
1,0008c4fab9e97623a60380ee9c88cb20,female,politician,left,left,"Feliz año nuevo, feliz esperanza 💕. Querido 20...","Feliz Año Nuevo, Feliz Esperanza :two_hearts: ..."
2,0008c4fab9e97623a60380ee9c88cb20,female,politician,left,left,🇩🇪🇪🇸 ¡Un placer encontrarme con mi homólogo al...,:Germany::Spain: ¡Un Placer Encontrarme con MI...
3,0008c4fab9e97623a60380ee9c88cb20,female,politician,left,left,El conflicto en Ucrania ha supuesto una dramát...,El Conflicto en Ucrania ha Supuesto Una Dramát...
4,0008c4fab9e97623a60380ee9c88cb20,female,politician,left,left,La Academia de la Llingua Asturiana realiza un...,La academia de la Llingua Asturiana Realiza UN...
...,...,...,...,...,...,...,...
179995,ffd89e81d6f6c783bfb72a4590db4304,male,politician,left,left,Desde un inicio nos opusimos a la escalda mili...,Desde UN Inicio nos Opusimos a La Escalda Mili...
179996,ffd89e81d6f6c783bfb72a4590db4304,male,politician,left,left,En menos de 4 minutos he tratado de analizar e...,En Menos de 4 Minutos he Tratado de Analizar E...
179997,ffd89e81d6f6c783bfb72a4590db4304,male,politician,left,left,Un fantasma recorre Euskal Herria y el Estado....,Un Fantasma Recorre Euskal Herria Y El Estado ...
179998,ffd89e81d6f6c783bfb72a4590db4304,male,politician,left,left,Aquí os dejo mis reflexiones hoy en el diario ...,Aquí os Dejo MIS Reflexiones Hoy en El Diario ...
